In [ ]:
import os
from pathlib import Path
from PIL import Image

from google.colab import drive
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision import models
from sklearn.metrics import accuracy_score

In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
# подключаем GDrive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d gpiosenka/100-bird-species


100% 1.49G/1.49G [00:14<00:00, 145MB/s]
100% 1.49G/1.49G [00:14<00:00, 114MB/s]


In [ ]:
!unzip 100-bird-species.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: train/VERMILION FLYCATHER/102.jpg  
  inflating: train/VERMILION FLYCATHER/103.jpg  
  inflating: train/VERMILION FLYCATHER/104.jpg  
  inflating: train/VERMILION FLYCATHER/105.jpg  
  inflating: train/VERMILION FLYCATHER/106.jpg  
  inflating: train/VERMILION FLYCATHER/107.jpg  
  inflating: train/VERMILION FLYCATHER/108.jpg  
  inflating: train/VERMILION FLYCATHER/109.jpg  
  inflating: train/VERMILION FLYCATHER/110.jpg  
  inflating: train/VERMILION FLYCATHER/111.jpg  
  inflating: train/VERMILION FLYCATHER/112.jpg  
  inflating: train/VERMILION FLYCATHER/113.jpg  
  inflating: train/VERMILION FLYCATHER/114.jpg  
  inflating: train/VERMILION FLYCATHER/115.jpg  
  inflating: train/VERMILION FLYCATHER/116.jpg  
  inflating: train/VERMILION FLYCATHER/117.jpg  
  inflating: train/VERMILION FLYCATHER/118.jpg  
  inflating: train/VERMILION FLYCATHER/119.jpg  
  inflating: train/VERMILION FLYCATHER/120.jpg  
 

In [ ]:
# !rm -rf /root/.kagle && rm -rf /root/.kaggle

In [ ]:
# !mkdir /root/.kaggle && mv kaggle.json /root/.kaggle

In [ ]:
# import kaggle

In [ ]:
data_path = Path(".")

In [ ]:
# распакуем файлы
# !cd /content/drive/MyDrive/hw2 && unzip -qq test.zip -d test && unzip -qq valid.zip -d valid && unzip -qq train.zip -d train

In [ ]:
# посмотрим на папки в train
#!ls /content/drive/MyDrive/hw2/archive/train

In [ ]:
# заведем путь до папки с данными
#data_path = Path("/content/drive/MyDrive/hw2")

In [ ]:
# пути до отдельных датасетов
train_data_path = data_path / "train"
valid_data_path = data_path / "valid"
test_data_path = data_path / "test"

In [ ]:
# поскольку мы предсказываем индекс класса а не его имя, необходимо завести mapping имя - индекс
# mapping строится на данных трейна, все остальные классы которые не присутствуют в нем будут трактоваться как -1
class_to_idx = {}
for i, directory in enumerate(os.listdir(train_data_path)):
    # YOUR CODE
    class_to_idx[directory] = i
    # b = class_to_idx.get(i, -1)

In [ ]:
class ImageDataset(Dataset):

    def __init__(self, file_path, transform, class_to_idx):
        self.files = list(file_path.rglob("*.jpg")) 
        self.transform = transform
        self.class_to_idx = class_to_idx
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        file = self.files[i]
        img = Image.open(file)
        tensor = self.transform(img)
        label = file.parent.name
        # берем имя родительской директории file https://docs.python.org/3/library/pathlib.html#pathlib.PurePath.parent
        label_id = self.class_to_idx.get(label, -1)
        # если label в self.class_to_idx то возращаем его id если нет то -1
        return tensor, label_id

In [ ]:
# используйте разные трансформации train чтобы для того чтобы данные были более различны можно почитать про AutoAgument https://pytorch.org/vision/stable/transforms.html#automatic-augmentation-transforms
train_transform = transforms.Compose([
    transforms.Resize(299),
    transforms.RandomCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], # Используем нормализацию изображений для ImageNet моделей
        std=[0.229, 0.224, 0.225]
    )
])

# трансформация на валидации и тесте остается неизменной

valid_transform = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
train_dataset = ImageDataset(train_data_path, train_transform, class_to_idx)
valid_dataset = ImageDataset(valid_data_path, valid_transform, class_to_idx)
test_dataset = ImageDataset(test_data_path, valid_transform, class_to_idx)

In [ ]:
# Определим параметры обучения

BATCH_SIZE = 64
EPOCH = 10
LEARNING_RATE = 0.0005
WEIGHT_DECAY = 1e-6

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)

In [ ]:
#net = models.alexnet(pretrained=True) # пробуйте разные модели, с предобучением на ImageNet и без
num_classes = 400
net = models.inception_v3(pretrained=False, num_classes = num_classes)
net = net.to("cuda")

/usr/local/lib/python3.7/dist-packages/torchvision/models/inception.py:83: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)


In [28]:
#optimizer = # Ваш любимый оптимизатор с параметрами LEARNING RATE и WEIGHT DECAY
optimizer = optim.Adam(net.parameters(), lr = LEARNING_RATE, weight_decay = WEIGHT_DECAY)

In [29]:
#criteria = # используем функцию потерь для многоклассовой классификации
criteria = nn.CrossEntropyLoss()

In [30]:

for epoch in range(EPOCH):
    
    net.train() # перевести сеть в режим тренировки
    for i, (x, y) in enumerate(train_loader):
        
        optimizer.zero_grad() # обнулить градиент оптимизатора
        x = x.to("cuda")
        y = y.to("cuda")# перевести x, y на cuda
        output = net.forward(x.float())# получить выход сети
        output = output.logits
        loss = criteria(output, y)# посчитать лосс
        loss.backward() # сделать backward propagation от лосса
        optimizer.step()  # шаг оптимизатора
        
        if (i + 1) % 100 == 0:
            print(f"Epoch {epoch + 1} / 10, iteration {i + 1} / {len(train_loader)}, loss: {loss.item():.3f}")
        
    y_true = []
    y_pred = []
    
    net.eval()   # перевести сеть в режим предсказаний
    for i, (x, y) in enumerate(valid_loader):
        # выключить градиент
        with torch.no_grad():
            x = x.to("cuda") # переводим x на cuda
            y = y.flatten() # переводим y в numpy массив
            
            output = net.forward(x.float())# считаем выход сети
            output = output.logits
            output = output.argmax(1).cpu().numpy() # берем самый вероятный класс и переводим output на cpu и в numpy массив
            y_true.extend(y)
            y_pred.extend(output)
            
    score = accuracy_score(y_true, y_pred)
    print(f"Epoch {epoch + 1} / 10, accuracy: {score:.3f}")

Epoch 1 / 10, iteration 100 / 913, loss: 5.663
Epoch 1 / 10, iteration 200 / 913, loss: 4.754
Epoch 1 / 10, iteration 300 / 913, loss: 4.893
Epoch 1 / 10, iteration 400 / 913, loss: 4.781
Epoch 1 / 10, iteration 500 / 913, loss: 4.173
Epoch 1 / 10, iteration 600 / 913, loss: 4.210
Epoch 1 / 10, iteration 700 / 913, loss: 3.774
Epoch 1 / 10, iteration 800 / 913, loss: 3.452
Epoch 1 / 10, iteration 900 / 913, loss: 3.145


AttributeError: ignored

In [ ]:
# считаем итоговое качество

for i, (x, y) in enumerate(test_loader):
    # выключить градиент
    with torch.no_grad():
        x = x.to("cuda") # переводим x на cuda
        y = y.flatten()# переводим y в numpy массив
        
        output = net.forward(x.float())# считаем выход сети
        output = output.logits.argmax(1).cpu().numpy()# берем самый вероятный класс и переводим output на cpu и в numpy массив
        y_true.extend(y)
        y_pred.extend(output)
        
score = accuracy_score(y_true, y_pred)
print(f"Test accuracy: {score:.3f}")